<a href="https://colab.research.google.com/github/tnc-br/ddf_common/blob/get_predictions/hypothesis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
TEST_OXYGEN_ISOSCAPE_MEANS_FILENAME = "canonical/kriging_random_means.tiff" #@param
TEST_OXYGEN_ISOSCAPE_VARS_FILENAME = "canonical/kriging_random_means.tiff" #@param

TEST_NITROGEN_ISOSCAPE_MEANS_FILENAME = "plant_nitrogen_isoscape.tiff" #@param
TEST_NITROGEN_ISOSCAPE_VARS_FILENAME = "plant_nitrogen_isoscape.tiff" #@param

In [2]:
#@title Imports and modules.
%pip install opencv-python
%pip install matplotlib
%pip install pandas

from osgeo import gdal, gdal_array
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
import matplotlib.animation as animation
from matplotlib import rc
from typing import List
from numpy.random import MT19937, RandomState, SeedSequence
import pandas as pd
from tqdm import tqdm
from io import StringIO
import xgboost as xgb
import os
import math
import glob

rc('animation', html='jshtml')

import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
ddfimport.ddf_source_control_pane()
# ddfimport.ddf_import_common()

interactive(children=(Text(value='', description='Email', placeholder='Enter email'), Text(value='', descripti…

# Tests

In [10]:
import pytest
import hypothesis
import raster

In [35]:
import importlib
import hypothesis
importlib.reload(hypothesis)

<module 'hypothesis' from '/content/gdrive/MyDrive/get_predictions/ddf_common/hypothesis.py'>

In [7]:
def sample_ttest_test():
  longitude = -71
  latitude = -5.5
  # The range of expected values is around 25 for most locations here.
  isotope_values = [-20.0, -15.0]
  test_means_isoscape = raster.load_raster(raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_MEANS_FILENAME), use_only_band_index=0)
  test_vars_isoscape = raster.load_raster(raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_VARS_FILENAME), use_only_band_index=0)
  sample_size_per_location = 2
  p_value_target = 0.05

  hypothesis_test = hypothesis.sample_ttest(longitude,
                 latitude,
                 isotope_values,
                 test_means_isoscape,
                 test_vars_isoscape,
                 sample_size_per_location,
                 p_value_target)

  assert(hypothesis_test.longitude == pytest.approx(-71))
  assert(hypothesis_test.latitude == pytest.approx(-5.5))
  assert(hypothesis_test.p_value == pytest.approx(0.014089625399513375))
  assert(hypothesis_test.p_value_threshold == pytest.approx(0.05))

sample_ttest_test()

Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)
Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)


In [8]:
import unittest

class SampleTTestCase(unittest.TestCase):
  def sample_ttest_test_tail_location(self):
    longitude = -71
    latitude = -5.5
    # The range of expected values is around 25 for most locations here.
    isotope_values = [-20.0]
    test_means_isoscape = raster.load_raster(raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_MEANS_FILENAME), use_only_band_index=0)
    test_vars_isoscape = raster.load_raster(raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_VARS_FILENAME), use_only_band_index=0)
    sample_size_per_location = 2
    p_value_target = 0.05

    with self.assertRaises(ValueError):
      hypothesis.sample_ttest(longitude,
                    latitude,
                    isotope_values,
                    test_means_isoscape,
                    test_vars_isoscape,
                    sample_size_per_location,
                    p_value_target)

SampleTTestCase().sample_ttest_test_tail_location()

Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)
Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)


In [17]:
def fraud_metrics_test():
  # The range of expected values is around 25 for most locations here.
  sample_data = pd.DataFrame({
      'Code': ["a", "b", "c", "d", "e", "f", "g", "h",
               "a", "b", "c", "d", "e", "f", "g", "h"],
      'long': [-71, -69, -68, -67, -66, -65, -64, -63,
               -71, -69, -68, -67, -66, -65, -64, -63],
      'lat': [-5.5, -5, -4.5, -4.3, -4, -3.9, -3.5, -3,
              -5.5, -5, -4.5, -4.3, -4, -3.9, -3.5, -3],
      'fraud': [True, False, True, False, True, False, True, False,
                True, False, True, False, True, False, True, False],
      'd18O_cel': [-20.0, 5.0, 100.0, 5.0, 8.0, 5.0, 0.0, 3.0,
                   -20.0, 25.0, 105.0, 25.0, 20.0, 2.0, 0.0, 25.0],
  })
  isotope_column_names = ['d18O_cel']
  test_means_isoscapes = [raster.load_raster(raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_MEANS_FILENAME), use_only_band_index=0)]
  test_vars_isoscapes = [raster.load_raster(raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_VARS_FILENAME), use_only_band_index=0)]
  sample_size_per_location = 2
  p_value_target = 0.05

  fraud_metrics = (
    hypothesis.fraud_metrics(sample_data,
                  isotope_column_names,
                  test_means_isoscapes,
                  test_vars_isoscapes,
                  sample_size_per_location,
                  p_value_target)
  )

  print(fraud_metrics.accuracy)
  print(fraud_metrics.precision)
  print(fraud_metrics.recall)
  assert(fraud_metrics.accuracy == pytest.approx(0.625))
  assert(fraud_metrics.precision == pytest.approx(1.0))
  assert(fraud_metrics.recall == pytest.approx(0.25))

fraud_metrics_test()

Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)
Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)
0.625
1.0
0.25


In [18]:
def fraud_metrics_combine_p_values_test():
  # The range of expected values is around 25 for most locations here.
  sample_data = pd.DataFrame({
      'Code': ["a", "b", "c", "d", "e", "f", "g", "h",
               "a", "b", "c", "d", "e", "f", "g", "h"],
      'long': [-71, -69, -68, -67, -66, -65, -64, -63,
               -71, -69, -68, -67, -66, -65, -64, -63],
      'lat': [-5.5, -5, -4.5, -4.3, -4, -3.9, -3.5, -3,
              -5.5, -5, -4.5, -4.3, -4, -3.9, -3.5, -3],
      'fraud': [True, False, True, False, True, False, True, False,
                True, False, True, False, True, False, True, False],
      'd18O_cel': [-20.0, 5.0, 100.0, 5.0, 8.0, 5.0, 0.0, 3.0,
                   -20.0, 25.0, 105.0, 25.0, 20.0, 2.0, 0.0, 25.0],
      'd15N_wood': [-20.0, 25.0, 105.0, 25.0, 20.0, 2.0, 0.0, 25.0,
                     -20.0, 5.0, 100.0, 5.0, 8.0, 5.0, 0.0, 3.0],
  })
  isotope_column_names = ['d18O_cel', 'd15N_wood']
  test_means_isoscapes = [
      raster.load_raster(
          raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_MEANS_FILENAME),
          use_only_band_index=0),
      raster.load_raster(
          raster.get_raster_path(TEST_NITROGEN_ISOSCAPE_MEANS_FILENAME),
          use_only_band_index=0)]
  test_vars_isoscapes = [
      raster.load_raster(
          raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_VARS_FILENAME),
          use_only_band_index=0),
      raster.load_raster(
          raster.get_raster_path(TEST_NITROGEN_ISOSCAPE_VARS_FILENAME),
          use_only_band_index=0)]
  sample_size_per_location = 2
  p_value_target = 0.05

  print("in test", isotope_column_names)
  fraud_metrics = (
    hypothesis.fraud_metrics(sample_data,
                  isotope_column_names,
                  test_means_isoscapes,
                  test_vars_isoscapes,
                  sample_size_per_location,
                  p_value_target)
  )

  assert(fraud_metrics.accuracy == pytest.approx(0.875))
  assert(fraud_metrics.precision == pytest.approx(1.0))
  assert(fraud_metrics.recall == pytest.approx(0.75))

fraud_metrics_combine_p_values_test()

Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)
Driver: GTiff/GeoTIFF
Size is 3671 x 2631 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-73.9872799892171, 5.26666984632233)
Pixel Size = (0.008333333767853947, -0.008333333767876788)
Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223

In [19]:
# Test that we skip locations with one measurement.
def fraud_metrics_some_tail_locations():
  # The range of expected values is around 25 for most locations here.
  # One of the (-67, -4.3) data points was removed. This element was a true
  # negative (non-fraud) so accuracy is affected (decreases) but precision
  # and recall weren't affected.
  sample_data = pd.DataFrame({
      'Code': ["a", "b", "c", "e", "f", "g", "h",
               "a", "b", "c", "d", "e", "f", "g", "h"],
      'long': [-71, -69, -68, -66, -65, -64, -63,
               -71, -69, -68, -67, -66, -65, -64, -63],
      'lat': [-5.5, -5, -4.5, -4, -3.9, -3.5, -3,
              -5.5, -5, -4.5, -4.3, -4, -3.9, -3.5, -3],
      'fraud': [True, False, True, True, False, True, False,
                True, False, True, False, True, False, True, False],
      'd18O_cel': [-20.0, 5.0, 100.0, 8.0, 5.0, 0.0, 3.0,
                   -20.0, 25.0, 105.0, 25.0, 20.0, 2.0, 0.0, 25.0],
  })
  isotope_column_name = ['d18O_cel']
  test_means_isoscape = [raster.load_raster(
      raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_MEANS_FILENAME),
      use_only_band_index=0)]
  test_vars_isoscape = [raster.load_raster(
      raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_VARS_FILENAME),
      use_only_band_index=0)]
  sample_size_per_location = 2
  p_value_target = 0.05

  fraud_metrics = (
    hypothesis.fraud_metrics(sample_data,
                  isotope_column_name,
                  test_means_isoscape,
                  test_vars_isoscape,
                  sample_size_per_location,
                  p_value_target)
  )


  print(fraud_metrics.accuracy, fraud_metrics.precision, fraud_metrics.recall)
  assert(fraud_metrics.accuracy == pytest.approx(0.5714285714285714))
  assert(fraud_metrics.precision == pytest.approx(1.0))
  assert(fraud_metrics.recall == pytest.approx(0.25))

fraud_metrics_some_tail_locations()

Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)
Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)
0.5714285714285714 1.0 0.25


In [24]:
# Test where all locations are unique.
def fraud_metrics_unique_locations():
  # The range of expected values is around 25 for most locations here.
  sample_data = pd.DataFrame({
      'Code': ["a", "b", "c", "d", "e", "f", "g", "h"],
      'long': [-71, -69, -68, -67, -66, -65, -64, -63],
      'lat': [-5.5, -5, -4.5, -4.3, -4, -3.9, -3.5, -3],
      'fraud': [True, False, True, False, True, False, True, False],
      'd18O_cel': [-20.0, 5.0, 100.0, 5.0, 8.0, 5.0, 0.0, 3.0],
  })
  isotope_column_name = ['d18O_cel']
  test_means_isoscape = [raster.load_raster(
      raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_MEANS_FILENAME),
      use_only_band_index=0)]
  test_vars_isoscape = [raster.load_raster(
      raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_VARS_FILENAME),
      use_only_band_index=0)]
  sample_size_per_location = 2
  p_value_target = 0.05

  fraud_metrics = (
    hypothesis.fraud_metrics(sample_data,
                  isotope_column_name,
                  test_means_isoscape,
                  test_vars_isoscape,
                  sample_size_per_location,
                  p_value_target)
  )

  assert(fraud_metrics.accuracy == pytest.approx(0))
  assert(fraud_metrics.precision == pytest.approx(0))
  assert(fraud_metrics.recall == pytest.approx(0))

fraud_metrics_unique_locations()

Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)
Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)


In [39]:
from pandas.util.testing import assert_frame_equal

def get_predictions_test():
  # The range of expected values is around 25 for most locations here.
  sample_data = pd.DataFrame({
      'Code': ["a", "b", "c", "d", "e", "f", "g", "h",
               "a", "b", "c", "d", "e", "f", "g", "h"],
      'long': [-71.0, -69.0, -68.0, -67.0, -66.0, -65.0, -64.0, -63.0,
               -71.0, -69.0, -68.0, -67.0, -66.0, -65.0, -64.0, -63.0],
      'lat': [-5.5, -5, -4.5, -4.3, -4, -3.9, -3.5, -3,
              -5.5, -5, -4.5, -4.3, -4, -3.9, -3.5, -3],
      'fraud': [True, False, True, False, True, False, True, False,
                True, False, True, False, True, False, True, False],
      'd18O_cel': [-20.0, 5.0, 100.0, 5.0, 8.0, 5.0, 0.0, 3.0,
                   -20.0, 25.0, 105.0, 25.0, 20.0, 2.0, 0.0, 25.0],
  })
  isotope_column_names = ['d18O_cel']
  test_means_isoscapes = [raster.load_raster(raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_MEANS_FILENAME), use_only_band_index=0)]
  test_vars_isoscapes = [raster.load_raster(raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_VARS_FILENAME), use_only_band_index=0)]
  sample_size_per_location = 2

  predictions = (
    hypothesis.get_predictions(sample_data,
                  isotope_column_names,
                  test_means_isoscapes,
                  test_vars_isoscapes,
                  sample_size_per_location)
  )

  expected_data = pd.DataFrame({
      'Code': ["a", "b", "c", "d", "e", "f", "g", "h"],
      'long': [-71.0, -69.0, -68.0, -67.0, -66.0, -65.0, -64.0, -63.0],
      'lat': [-5.5, -5, -4.5, -4.3, -4, -3.9, -3.5, -3],
      'fraud': [True, False, True, False, True, False, True, False],
      'predicted_fraud': [0.050020, 0.520953, 0.004640, 0.503469, 0.284436, 0.067667,
                          0.088811, 0.474167]
  })
  assert_frame_equal(predictions.reset_index(drop=True),
                     expected_data.reset_index(drop=True), check_dtype=False,
                     check_exact=False, atol=1e-4)

get_predictions_test()

Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)
Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)


/content/gdrive/MyDrive/get_predictions/ddf_common/hypothesis.py:131: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  predictions = pd.concat([predictions, pd.DataFrame(row)], ignore_index=True)


In [41]:
# Test that we skip locations with one measurement.
def get_predictions_some_tail_locations():
  # The range of expected values is around 25 for most locations here.
  # One of the (-67, -4.3) data points was removed. This element was a true
  # negative (non-fraud) so accuracy is affected (decreases) but precision
  # and recall weren't affected.
  sample_data = pd.DataFrame({
      'Code': ["a", "b", "c", "e", "f", "g", "h",
               "a", "b", "c", "d", "e", "f", "g", "h"],
      'long': [-71, -69, -68, -66, -65, -64, -63,
               -71, -69, -68, -67, -66, -65, -64, -63],
      'lat': [-5.5, -5, -4.5, -4, -3.9, -3.5, -3,
              -5.5, -5, -4.5, -4.3, -4, -3.9, -3.5, -3],
      'fraud': [True, False, True, True, False, True, False,
                True, False, True, False, True, False, True, False],
      'd18O_cel': [-20.0, 5.0, 100.0, 8.0, 5.0, 0.0, 3.0,
                   -20.0, 25.0, 105.0, 25.0, 20.0, 2.0, 0.0, 25.0],
  })
  isotope_column_name = ['d18O_cel']
  test_means_isoscape = [raster.load_raster(
      raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_MEANS_FILENAME),
      use_only_band_index=0)]
  test_vars_isoscape = [raster.load_raster(
      raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_VARS_FILENAME),
      use_only_band_index=0)]
  sample_size_per_location = 2
  p_value_target = 0.05

  predictions = (
    hypothesis.get_predictions(sample_data,
                  isotope_column_name,
                  test_means_isoscape,
                  test_vars_isoscape,
                  sample_size_per_location)
  )

  expected_data = pd.DataFrame({
      'Code': ["a", "b", "c", "e", "f", "g", "h"],
      'long': [-71.0, -69.0, -68.0, -66.0, -65.0, -64.0, -63.0],
      'lat': [-5.5, -5, -4.5, -4, -3.9, -3.5, -3],
      'fraud': [True, False, True, True, False, True, False],
      'predicted_fraud': [0.050020, 0.520953, 0.004640, 0.284436, 0.067667,
      0.088811, 0.474167]
  })
  assert_frame_equal(predictions.reset_index(drop=True),
                     expected_data.reset_index(drop=True), check_dtype=False,
                     check_exact=False, atol=1e-4)

get_predictions_some_tail_locations()

Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)
Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)


/content/gdrive/MyDrive/get_predictions/ddf_common/hypothesis.py:131: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  predictions = pd.concat([predictions, pd.DataFrame(row)], ignore_index=True)


In [43]:
# Test where all locations are unique.
def get_predictions_unique_locations():
  # The range of expected values is around 25 for most locations here.
  sample_data = pd.DataFrame({
      'Code': ["a", "b", "c", "d", "e", "f", "g", "h"],
      'long': [-71, -69, -68, -67, -66, -65, -64, -63],
      'lat': [-5.5, -5, -4.5, -4.3, -4, -3.9, -3.5, -3],
      'fraud': [True, False, True, False, True, False, True, False],
      'd18O_cel': [-20.0, 5.0, 100.0, 5.0, 8.0, 5.0, 0.0, 3.0],
  })
  isotope_column_name = ['d18O_cel']
  test_means_isoscape = [raster.load_raster(
      raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_MEANS_FILENAME),
      use_only_band_index=0)]
  test_vars_isoscape = [raster.load_raster(
      raster.get_raster_path(TEST_OXYGEN_ISOSCAPE_VARS_FILENAME),
      use_only_band_index=0)]
  sample_size_per_location = 2
  p_value_target = 0.05

  predictions = (
    hypothesis.get_predictions(sample_data,
                  isotope_column_name,
                  test_means_isoscape,
                  test_vars_isoscape,
                  sample_size_per_location)
  )

  expected_data = pd.DataFrame({
      'Code': [],
      'long': [],
      'lat': [],
      'fraud': [],
      'predicted_fraud': []
  })
  assert_frame_equal(predictions.reset_index(drop=True),
                     expected_data.reset_index(drop=True))

get_predictions_unique_locations()

Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)
Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)
